# Julia Set Plotting Extension

Load module for a JuliaSet that conforms to the specified interface.

It is wise to run the test suite in `test_juliaset.py` with `nosetests` prior to attempting to plot here.

In [ ]:
from juliaset import JuliaSet

Load additional libraries needed for plotting and profiling.

In [ ]:
# Math libraries
import numpy as np
from math import sqrt

# Matplotlib plotting libraries
import matplotlib.pyplot as plt
%matplotlib inline

# Bokeh plotting libraries
import bokeh.plotting as blt
blt.output_notebook()

Extend JuliaSet class with additional functionality.

In [ ]:
class JuliaSetPlot(JuliaSet):
    """Extend JuliaSet to add plotting functionality"""
    
    def __init__(self, *args, **kwargs):
        # Invoke constructor for JuliaSet first, unaltered
        JuliaSet.__init__(self, *args, **kwargs)
        # Add another attribute: a rendered image array
        self.img = np.array([])
    
    def get_dim(self):
        """Return linear number of points in axis"""
        return int(4.0 / self._d)
    
    def render(self):
        """Render image as square array of ints"""
        if not self.set: self.generate()
        # Convert inefficient list to efficient numpy array
        self.img = np.array(self.set)
        # Reshape array into a 2d complex plane
        dim = int(sqrt(len(self.img)))
        self.img = np.reshape(self.img, (dim,dim)).T
        
    def show(self):
        """Use matplotlib to plot image as an efficient mesh"""
        if not self.img.size: self.render()
        plt.figure(1, figsize=(12,9))
        xy = np.linspace(-2,2,self.get_dim())
        plt.pcolormesh(xy, xy, self.img, cmap=plt.cm.hot)
        plt.colorbar()
        plt.show()
        
    def interact(self):
        """Use bokeh to plot an interactive image"""
        from matplotlib.colors import rgb2hex
        if not self.img.size: self.render()
        # Mimic matplotlib "hot" color palette
        colormap = plt.cm.get_cmap("hot")
        bokehpalette = [rgb2hex(m) for m in colormap(np.arange(colormap.N))]
        # Create bokeh figure
        f = blt.figure(x_range=(-2,2), y_range=(-2,2), plot_width=600, plot_height=600)
        f.image(image=[self.img], x=[-2], y=[-2], dw=[4], dh=[4], palette=bokehpalette, dilate=True)
        blt.show(f)

Visualize a Julia set using matplotlib.

In [ ]:
j = JuliaSetPlot(-1.037 + 0.17j)
%time j.set_spacing(0.006)
%time j.show()

Visualize a different Julia set using Bokeh as an interactive Javascript plot.

In [ ]:
j = JuliaSetPlot(-0.624 + 0.435j)
%time j.set_spacing(0.006)
%time j.interact()

In [ ]:
%prun j.generate()

In [ ]:
%load_ext line_profiler
%lprun -f j.generate j.generate()